<a href="https://colab.research.google.com/github/DataTak/study_deeplearning_fromscratch/blob/main/7%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 합성곱 신경망(Convolutional Neural Network)

## 개념

- 완전연결계층(Fully Connected)<br>
![완전연결계층](https://cael0.github.io/assets/images/2021_08_16/7_1_1.PNG)
- CNN 네트워크 예시<br>
![CNN](https://cael0.github.io/assets/images/2021_08_16/7_1_2.PNG)
-출처: https://cael0.github.io/deep%20learning/DeepLearningCh7/

## 합성곱계층
- 완전연결 계층의 문제점
    - 데이터의 형상이 무시된다.
        - 3차원 데이터 => 1차원으로 평탄화 해서 입력
        - 공간적 정보가 사라짐. 모든 입력 데이터를 동등한 뉴런으로 취급하여 형상에 담긴 정보를 살릴 수 없음.
- 합성곱 계층
    - 형상을 유지
    - 입출력 데이터를 특징 맵(feature map) 이라고도 부름.


![합성곱 연산 계산 순서](https://cael0.github.io/assets/images/2021_08_16/7_2_2_3.PNG)
- 데이터의 형상 = (높이, 너비) = (행 개수, 열 개수)

- 입력 : (4, 4) / 필터 (커널) : (3, 3) / 출력 : (2, 2)

- 윈도우 (window) : 필터가 입력 데이터와 겹치는 부분

- 단일 곱셈-누산 (fused multiply-add, FMA) : 대응하는 원소끼리 곱한 후 총합을 구하는 계산

- 출처: 출처: https://cael0.github.io/deep%20learning/DeepLearningCh7/

### 패딩

- Padding(패딩)
    - 합성곱 연산 수행 전 입력 데이터 주변을 특정 값으로 채우는 것.
    - 패딩은 출력 크기를 조절할 목적으로 사용
        - (4,4) 입력 데이터에 (3,3) 필터를 적용하면 출력은 (2,2)
        - 합성곱 연산을 거칠 때 마다 크기가 작아지면 어느순간 출력 크기가 1.

### 스트라이드

- Stride(스트라이드):  필터를 적용하는 위치의 간격<br>
![스트라이드](https://cael0.github.io/assets/images/2021_08_16/7_2_4.PNG)


- 스트라이드를 크게 하면 => 출력의 크기는 작아진다.
- 패딩을 크게 하면 => 출력의 크기가 커진다.<br>
- 입력 크기(H,W), 필터 크기(FH, FW), 출력 크기(OH, OW), 패딩 P, 스트라이드 S 일 때 출력의 크기
- $
O_H = \frac{H + 2P - F_H}{S} + 1
$

- $
O_W = \frac{W + 2P - F_W}{S} + 1
$


### 3차원 합성곱 신경망
![3차원합성곱](https://cael0.github.io/assets/images/2021_08_16/7_2_5_2.PNG)
- 출처는 기존과 동일
- 연산시 주의점
    - 입력 데이터의 채널 수와 필터의 채널수가 같아야 함.
    - 모든 채널의 필터 크기는 같아야 함.<br>
![여러 필터를 사용한 합성곱연산](https://cael0.github.io/assets/images/2021_08_16/7_2_6_2.PNG)

### 배치처리
![배치처리](https://cael0.github.io/assets/images/2021_08_16/7_2_7.PNG)
- 4차원으로 저장(데이터의 수, 채널 수, 높이, 너비)
- 신경망에 4차원 데이터가 하나 흐를 때마다 데이터 N개에 대한 합성곱 연산이 이뤄진다.(N회 분의 처리를 한 번에 수행)

## 풀링계층

![풀링](https://cael0.github.io/assets/images/2021_08_16/7_3.PNG)
- 2*2 최대풀링(max pooling)을 스트라이드 2로 처리하는 순서
- 평균 풀링도 있는데 이미지 인식에서는 주로 맥스풀링 사용

- 풀링 계층의 특징
    - 학습해야 할 매개변수가 없다.
    - 채널 수가 변하지 않는다.
    - 입력의 변화에 영향을 적게 받는다(강건하다).

## 합성곱/풀링 계층 구현

In [3]:
# 4차원 배열
import numpy as np
x = np.random.rand(10, 1, 28, 28)  #(데이터수,  채널, 높이, 너비)
x.shape

(10, 1, 28, 28)

In [5]:
x[0].shape

(1, 28, 28)

In [7]:
x[0, 0].shape

(28, 28)

## im2col
- image to column의 약자. (이미지에서 행렬로!)<br>
![im2col](https://cael0.github.io/assets/images/2021_08_16/7_4_2_3.PNG)

In [8]:
# 저자 깃 저장소 클론
!git clone https://github.com/WegraLee/deep-learning-from-scratch.git

Cloning into 'deep-learning-from-scratch'...
remote: Enumerating objects: 853, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 853 (delta 9), reused 13 (delta 5), pack-reused 830 (from 1)
Receiving objects: 100% (853/853), 52.33 MiB | 34.44 MiB/s, done.
Resolving deltas: 100% (486/486), done.


In [9]:
import sys, os
sys.path.append("/content/deep-learning-from-scratch")  #불러온 깃 저장소를
from dataset.mnist import load_mnist

In [10]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """
    다수의 이미지를 입력받아 2차원 배열로 변환 (평탄화)

    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩

    Returns
    -------
    col : 2차원 배열
    """

    N, C, H, W = input_data.shape
    out_h = (H + 2 * pad - filter_h) // stride + 1
    out_w = (W + 2 * pad - filter_w) // stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)
    return col

In [11]:
x1 = np.random.rand(1, 3, 7, 7)  # (데이터 수, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


#### 출력결과 이해

## 이미지와 필터 정보
1. **입력 이미지 크기**: $3 \times 7 \times 7$
   - $3$: 채널 수 (예: RGB)
   - $7 \times 7$: 각 채널의 높이와 너비
2. **필터 크기**: $5 \times 5$
   - 각 채널에서 $5 \times 5 = 25$개의 픽셀을 커버합니다.
3. **스트라이드**: $1$
   - 필터가 한 픽셀씩 움직이며 $3 \times 3 = 9$개의 윈도우를 생성합니다.

---

## `im2col`의 동작
1. **윈도우 개수**:
   - $3 \times 3 = 9$: 출력 특징 맵의 크기 (윈도우의 개수).
2. **각 윈도우의 데이터**:
   - 필터는 각 채널에서 $5 \times 5 = 25$개의 픽셀을 가져옵니다.
   - $3$개의 채널 데이터를 결합하므로, $25 \times 3 = 75$개의 데이터를 하나의 윈도우에 포함합니다.
3. **최종 결과**:
   - $9$개의 윈도우 각각에서 $75$개의 데이터를 가지므로, `im2col`의 결과는 $(9, 75)$ 크기를 가집니다.

---


In [12]:
x2 = np.random.rand(10, 3, 7, 7)  # (데이터 수, 채널 수, 높이, 너비)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)  # (90, 75)

(90, 75)


### 합성곱층 구현

In [13]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_w = self.W.reshape(FN, -1).T
        out = np.dot(col, col_w) + self.b

        out = out.reshpae(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        return out

- 4차원 입력 데이터 x를 im2col 함수를 통해 2차원으로 만들어주고, 이 행렬과 곱할 수 있도록 필터 W의 형상을 바꿔줌
- 행렬 곱 후 편향을 더한 결과를 다시 4차원으로 만들어주고, 축의 순서를 원래대로 변경함

### 풀링층 구현

![1](https://cael0.github.io/assets/images/2021_08_16/7_4_4_1.PNG)<br>
![2](https://cael0.github.io/assets/images/2021_08_16/7_4_4_2.PNG)

In [14]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshpae(-1, self.pool_h * self.pool_w)

        out = np.max(col, axis = 1)

        out = out.reshpae(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

1. 입력 데이터 전개
2. 행별 최댓값 구하기
3. 적절한 모양 성형

## CNN 구현

![1](https://cael0.github.io/assets/images/2021_08_16/7_5.PNG)

In [15]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28),
                conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['strid']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2 * filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size / 2) ** 2)

        # 가중치 매개변수 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0],
                                                             filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                            conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)

        layes = reversed(self.layers.values())
        for layer in layers:
            dout = layer.backward(dout)

        # 기울기 저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db

        return grads

## CNN 시각화하기

![1](https://cael0.github.io/assets/images/2021_08_16/7_6_1_1.PNG)

- 학습 전 필터는 무작위로 초기화 되어 있어 흑백의 정도에 규칙성이 없음
- 학습을 마친 필터는 규칙성 있는 이미지가 되었음.
- ![2](https://cael0.github.io/assets/images/2021_08_16/7_6_1_2.PNG)

### 층 깊이에 따른 추출 정보 변화
![1](https://cael0.github.io/assets/images/2021_08_16/7_6_2.PNG)

- 합성곱 계층을 여러 겹 쌓으면 층이 깊어지면서 더 복잡하고 추상회된 정보가 추출됨.
- 처음 층은 단순한 에지에 반응, 텍스처 반응, 복잡한 사물의 일부에 반응..
- 층이 깊어지면서 뉴런이 반응하는 대상이 단순한 모양에서 고급 정보로 변화해 감.(사물의 의미 이해)

## 대표적인 CNN
- LeNet
    - 손글씨 숫자 인식 네트워크(1988년 제안)
    - 활성화 함수로 시그모이드 사용
    - 서브샘플링을 하여 중간 데이터의 크기가 작아짐
- AlexNet
    - 2012년 발표
    - 합성곱 계층과 풀링 계층을 거듭하며 마지막으로 완전연결계층을 거쳐 결과 출력
    - 활성화 함수로 ReLU 사용
    - LRN(Local Response Normalization)이라는 국소적 정규화를 실시하는 계층 이용
    - 드롭아웃 사용

## 정리
- CNN은 지금까지 완전연결 계층 네트워크에 합성곱 계층과 풀링 계층을 새로 추가한다.
- 합성곱 계층과 풀링 계층은 im2col(이미지를 행렬로 전개하는 함수)를 이용하면 간단하고 효율적으로 구현할 수 있다.
- CNN을 시각화해보면 계층이 깊어질수록 고급 정보가 추출되는 모습을 확인할 수 있다.
- 대표적인 CNN에는 LeNet과 AlexNet이 있다.
- 딥러닝의 발전에는 빅 데이터와 GPU가 크게 기여했다.